In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests
import pandas as pd
import requests
import pymysql

# SQL Alchemy
import sqlalchemy
from sqlalchemy import create_engine, inspect, func 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import numpy as np

# import api_key from config file
from config import *

In [2]:

csv_file = "int_suicide_data.csv"
country_data_df = pd.read_csv(csv_file)
#country_data_df.head()   # Debug statement

In [3]:
#EF edit--dropped 'sex' column
new_country_data = country_data_df[['country', 'year', 'age', 'suicides_no']].copy()
#new_country_data.head()   # Debug statement

In [4]:
US_suicide = new_country_data[new_country_data["country"]== "United States"]
#US_suicide.head()   # Debug statement

In [5]:
age_suicide = US_suicide[US_suicide["age"]=="15-24 years"]

#age_suicide.head()   #Debug statement

In [6]:
year_suicide = age_suicide[(age_suicide.year == 1999) | (age_suicide.year == 2000) | (age_suicide.year == 2001)| (age_suicide.year == 2002)| (age_suicide.year == 2003)
           | (age_suicide.year == 2004)| (age_suicide.year == 2005)| (age_suicide.year == 2006)| (age_suicide.year == 2007)
           | (age_suicide.year == 2008)| (age_suicide.year == 2009)| (age_suicide.year == 2010)| (age_suicide.year == 2011)
           | (age_suicide.year == 2012)| (age_suicide.year == 2013)| (age_suicide.year == 2014)| (age_suicide.year == 2015)
           | (age_suicide.year == 2016)]

#year_suicide   #Debug statement

In [7]:
groupby_year_df = year_suicide.groupby(['year']).sum()
#groupby_year_df   #Debug statement

In [8]:
########################################
# SQLAlchemy for international data CSV
########################################


engine = create_engine("mysql+pymysql://root:"+ pw + "@localhost/")
conn = engine.connect()

In [9]:
drop_db = 'DROP DATABASE IF EXISTS suicide_data_db;'
engine.execute(drop_db)

C:\Users\aisha\Anaconda3\envs\pythondata2\lib\site-packages\pymysql\cursors.py:170: Warning: (1008, "Can't drop database 'suicide_data_db'; database doesn't exist")
  result = self._query(query)


In [10]:
#Create DB
create_db = 'CREATE DATABASE suicide_data_db;'
use_db = 'USE suicide_data_db;'

engine.execute(create_db)
engine.execute(use_db)


In [11]:
# Drop table suicide_data_db.suicide_data_us if exists already
drop_cdt = 'DROP TABLE IF EXISTS intl_suicide_data;'
engine.execute(drop_cdt)

# Create table suicide_data_db.year_suicide from year_suicide dataframe
groupby_year_df.to_sql('intl_suicide_data', con=engine)

C:\Users\aisha\Anaconda3\envs\pythondata2\lib\site-packages\pymysql\cursors.py:170: Warning: (1051, "Unknown table 'suicide_data_db.intl_suicide_data'")
  result = self._query(query)


In [12]:
###### Data scrape html file of US CDC Suicide Data Per State ######

In [13]:
# URL of page to be scraped
url = "CDC_Suicide_Query.html"

In [14]:
#Convert html table to pandas table
#the tables are returned as a list. It gives you a list of DFs (near the top) generated from the table elements in the html. 

tables = pd.read_html(url)
#tables   # Debug statement

In [15]:
df = tables[0]
#df   #Debug statement

In [16]:
#List of DF columns
#list(df.columns.values)   #Debug statement

In [17]:
clean_df = df[['Year', 'Unnamed: 2_level_0', 'State', 'Single-Year Ages', 'Deaths']]
clean_df.columns = ['Year', 'State', 'Age', 'Number_of_Suicides', 'Population_of_Age_Group', 'Something', 'Something2']
#clean_df   # Debug statement

In [18]:
cleaner_df = clean_df.drop(['Something', 'Something2'], 1)
cleaner_df['State'] = cleaner_df['State'].str[:-5]
#cleaner_df # Debug statement

In [19]:
#cleaner_df.dtypes   #Debug statement

In [20]:
######################################################
# SQLAlchemy for US CDC suicide data from HTML scraper
######################################################

In [21]:
# drop table suicide_data_db.suicide_data_us if exists already
drop_cdt = 'DROP TABLE IF EXISTS suicide_data_us;'
engine.execute(drop_cdt)

# create table suicide_data_db.suicide_data_us from cleaner_df dataframe
cleaner_df.to_sql('suicide_data_us', con=engine)

C:\Users\aisha\Anaconda3\envs\pythondata2\lib\site-packages\pymysql\cursors.py:170: Warning: (1051, "Unknown table 'suicide_data_db.suicide_data_us'")
  result = self._query(query)


In [22]:
##### Combine data from international CSV and US CDC html scraper #####

# Drop table suicide_data_db.combined_suicide_data if exists already
drop_combined_table = 'DROP TABLE IF EXISTS combined_suicide_data;'

# Create the table to combine the data
create_combined_table = 'CREATE TABLE IF NOT EXISTS combined_suicide_data (\
    state VARCHAR(255),\
    year BIGINT(20), \
    suicides_per_state BIGINT(20),\
    total_suicides_for_year BIGINT(20),\
    percent DECIMAL(5,3)\
) ENGINE=INNODB;'

# Insert into combined table summary of US state data
insert_us = 'INSERT INTO combined_suicide_data (state, year, suicides_per_state)\
    SELECT State, Year, sum(Number_of_Suicides) FROM suicide_data_db.suicide_data_us\
    GROUP BY State, Year\
    ORDER BY Year;'

# Update combined table to include world suicide data per year
insert_intl = 'UPDATE combined_suicide_data c, intl_suicide_data i\
    SET c.total_suicides_for_year = i.suicides_no\
    WHERE c.year = i.year;'

# Calculate percentage of suicide per state 
percentage = 'UPDATE combined_suicide_data\
    SET percent = ROUND(suicides_per_state/total_suicides_for_year,3);'

# Execute mySql procedures
engine.execute(drop_combined_table)
engine.execute(create_combined_table)
engine.execute(insert_us)
engine.execute(insert_intl)
engine.execute(percentage)

C:\Users\aisha\Anaconda3\envs\pythondata2\lib\site-packages\pymysql\cursors.py:170: Warning: (1051, "Unknown table 'suicide_data_db.combined_suicide_data'")
  result = self._query(query)
